In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import keras
from src.nimbus_analysis import autoencoder


2025-11-28 23:43:10.235201: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-28 23:43:10.263942: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-28 23:43:10.750009: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
with open('./datasets/train_X.pickle', 'rb') as file:
    train_X = pickle.load(file)
    
    
#rescale
train_X = train_X.astype('float32')/np.max(train_X)

#impute zero with median (zero exclusive)
nonzero_median = np.median(train_X[train_X > 0])
train_X[train_X == 0] = nonzero_median

train_y = train_X

#shift by 1 day between input and output
train_y = train_y[1:]
train_X = train_X[:-1]

In [3]:


autoencoder = autoencoder.global_conv_autoencoder()

W0000 00:00:1764373392.755429   18225 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder.fit(train_X, train_y,
                epochs=10,
                shuffle=False)

Epoch 1/2
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 103ms/step - loss: 0.0062
Epoch 2/2
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 102ms/step - loss: 0.0015


In [5]:
autoencoder.summary()

Model: "global_conv_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 45, 72, 8)      │         6,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 180, 288, 1)    │        14,873 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,469 (251.84 KB)

 Trainable params: 21,489 (83.94 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 42,980 (167.89 KB)

In [6]:
autoencoder.save('global_autoencoder.keras')